In [1]:
compressed_roster = "../data/roster.csv.zst"
compressed_teams = "../data/teams.csv.zst"
shortest_path_filename = "first_game_distances.csv"
bacon_numbers_filename = "bacon_numbers.csv"
teams_filename = "teams.csv"

In [2]:
import igraph as ig